In [1]:
# Import your library here

# Models for classifiying isCanerous status

Add the description for the following statements: 
- What is the problem to solve here? Binary classifcaiton
- Which metric should I used? (accuracy? weighted F1 score since we are having imbalanced dataset here, we should consider this)
- What is the baseline methods?
- Which models (**from the paper**) are used to compare with our model?